In [13]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn import metrics
import matplotlib.pyplot as plt
from functools import reduce
import math
from collections import Counter
import csv
from datetime import datetime
import re
%matplotlib inline

In [14]:
def now():
    return str(datetime.now().time())[:8]
def pr(strToPrint):
    print(now() + ' '+ strToPrint)

from IPython.display import Audio
sound_file = 'beep.wav'
# Used by other groups: pd.read_csv(data_path, sep="\t",encoding='utf-8',  escapechar='\\', quoting=csv.QUOTE_NONE, header=None, na_values='N')

In [15]:
pickle_filename = os.path.join('data','head_100k_pickle.pkl')
tw = pd.read_pickle(pickle_filename)

In [16]:
tw.head()

,id,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation
0,9514097914,17341045.0,2010-02-23 05:55:51,Guuuuten Morgen! :-),7.43926,46.9489,NaN,NaN,197.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,TwitBird,http://www.nibirutech.com,Tilman Jentzsch,blickwechsel,586.0,508.0,9016.0,"Bern, Switzerland",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9514846412,7198282.0,2010-02-23 06:22:40,Still the best coffee in town — at La Stanza h...,8.53781,47.3678,NaN,NaN,550.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Gowalla,http://gowalla.com/,Nico Luchsinger,halbluchs,1820.0,703.0,4687.0,"Zurich, Switzerland",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9516574359,14657884.0,2010-02-23 07:34:25,It has been a week or so.. and today I just co...,6.13396,46.1951,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
def extract_hashtags(text):
    ht_list = re.findall(r"#(\w+)", text)
    non_empty_hts = list(filter((lambda ht: ht != []), ht_list))
    return non_empty_hts

In [21]:
tw['hashtag'] = np.nan
tw.hashtag = tw.text.apply(lambda x: extract_hashtags(str(x)))
twh = tw.ix[tw.hashtag.apply(lambda x: len(x) != 0)]

In [ ]:
for i in range(twh):
    twh.iloc[i][]

In [22]:
twh.head()

,id,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,truncated,...,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation,hashtag
16,9519737890,14657884.0,2010-02-23 09:59:41,"Magic spells run off after midnight, I guess s...",6.1387,46.175,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[fb]
20,9521789689,9962022.0,2010-02-23 11:28:27,"Limitas of public transportation! No taxi, rai...",6.33641,46.4631,NaN,NaN,550.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[yam]
30,9535390586,921241.0,2010-02-23 17:47:11,"So, Feierabend. Jetzt #24 und später #VfB. — a...",9.1657,47.6463,NaN,NaN,550.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[24, VfB]"
32,9536575795,14260616.0,2010-02-23 18:19:03,Greetings ! http://tinyurl.com/ycrdlhq #iPhone...,8.2601,47.4576,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[iPhoneography]
34,9537030723,14542024.0,2010-02-23 18:31:46,Lexmark launches Evernote SmartSolution! Send ...,8.51865,47.3703,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[partnermonth]


In [25]:
# twh.hashtag.apply(lambda x: print(x) if len(x) == 0 else 1)

In [26]:
tw1 = twh.dropna(axis=0, how='any', subset=['text', 'createdAt'])
print('The data have been reduced from {} tweets to {} tweets.'.format(len(twh), len(tw1)))

The data have been reduced from 79200 tweets to 79200 tweets.


In [27]:
pr('Removing bad dates...')
twhCleanDate = tw1[tw1['createdAt'].str.len() == 19]
pr('Finished.')

10:32:06 Removing bad dates...
10:32:06 Finished.


In [28]:
pr('Starting to examine dates...')
import warnings
warnings.filterwarnings('ignore')
datetime_serie = twhCleanDate['createdAt'].convert_objects(convert_dates='coerce')
dateNotConvertible = datetime_serie[pd.isnull(datetime_serie)]
warnings.filterwarnings('default')
pr('There are {} dates that cannot be transformed.'.format(len(dateNotConvertible)))

10:32:06 Starting to examine dates...
10:32:07 There are 0 dates that cannot be transformed.


In [129]:
pr('Starting copy...') # (to avoid transformation problems)
tw5 = twhCleanDate.copy()
pr('Converting to datetime...')
tw5['createdAt'] = pd.to_datetime(twhCleanDate['createdAt'])
pr('Setting up new indices...')
tw5.index = tw5['createdAt']
pr('Deleting old "createdAt" column...')
del tw5['createdAt']
pr('Done!')
tw5.head(2)

13:39:17 Starting copy...
13:39:18 Converting to datetime...
13:39:18 Setting up new indices...
13:39:18 Deleting old "createdAt" column...
13:39:18 Done!


,id,userId,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation,hashtag
createdAt,,,,,,,,,,,,,,,,,,,,
2010-02-23 09:59:41,9519737890,14657884.0,"Magic spells run off after midnight, I guess s...",6.1387,46.175,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[fb]
2010-02-23 11:28:27,9521789689,9962022.0,"Limitas of public transportation! No taxi, rai...",6.33641,46.4631,NaN,NaN,550.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[yam]


In [130]:
tw5['hashtag']

createdAt
2010-02-23 09:59:41                             [fb]
2010-02-23 11:28:27                            [yam]
2010-02-23 17:47:11                        [24, VfB]
2010-02-23 18:19:03                  [iPhoneography]
2010-02-23 18:31:46                   [partnermonth]
2010-02-24 06:09:23                    [insider, fb]
2010-02-24 07:33:49                          [Blick]
2010-02-24 12:58:35                            [WIN]
2010-02-25 04:50:41                           [wach]
2010-02-25 04:56:37            [bwin, vancouver2010]
2010-02-25 08:47:28    [Tumblr, Betterthanwordpress]
2010-02-25 09:49:43                    [iPhone, PvZ]
2010-02-25 13:15:00                             [fb]
2010-02-25 20:54:38    [hierhinkommichwieder, essen]
2010-02-25 22:01:31                             [fb]
2010-02-26 04:56:51                   [nichtversteh]
2010-02-26 05:13:54     [frauen, maenner, beziehung]
2010-02-26 16:47:49                            [BCN]
2010-02-26 17:12:07                 

In [131]:
twt = tw5.head()
twt.hashtag

createdAt
2010-02-23 09:59:41               [fb]
2010-02-23 11:28:27              [yam]
2010-02-23 17:47:11          [24, VfB]
2010-02-23 18:19:03    [iPhoneography]
2010-02-23 18:31:46     [partnermonth]
Name: hashtag, dtype: object

In [135]:
columns = [ 'id', 'text', 'hashtag', 'longitude', 'latitude']
unique_ht_df =  pd.DataFrame(columns=columns)
unique_ht_df

,id,text,hashtag,longitude,latitude


In [32]:
tw5.hashtag = tw5.hashtag.apply(lambda x: ','.join(x).lower())
tw5.hashtag.head()

createdAt
2010-02-23 09:59:41               fb
2010-02-23 11:28:27              yam
2010-02-23 17:47:11           24,vfb
2010-02-23 18:19:03    iphoneography
2010-02-23 18:31:46     partnermonth
Name: hashtag, dtype: object

In [33]:
tw5.dropna(subset=['longitude'], inplace=True)
tw5.dropna(subset=['latitude'], inplace=True)

In [ ]:
columns = tw5.columns


In [127]:
tw5.head(1)
for i in range(len(tw5)):
    tw5.iloc[i][]

,id,userId,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation,hashtag
createdAt,,,,,,,,,,,,,,,,,,,,
2010-02-23 09:59:41,9519737890,14657884.0,"Magic spells run off after midnight, I guess s...",6.1387,46.175,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fb


In [34]:
tw5.latitude = tw5.latitude.apply(float)
tw5.longitude = tw5.longitude.apply(float)

In [36]:
delimiter = '_$$$_'
str_join = lambda x: delimiter.join(x)

In [39]:
def aggDate(df):
    test = df.groupby(df.index.map(lambda x: x.date)).agg({'text' : str_join,
                                                          'longitude' : np.median,
                                                          'latitude' : np.median,
                                                          'hashtag' : str_join})
    test['numberOfTweets'] = test['hashtag'].apply(lambda x: len(x.split(delimiter)))
    return test

In [40]:
gp = tw5.groupby('hashtag')
dictionary = {hashtag : aggDate(df) for hashtag, df in gp}

In [41]:
gp.get_group('hollande')

,id,userId,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation,hashtag
createdAt,,,,,,,,,,,,,,,,,,,,
2012-01-23 07:59:09,161357287192133632,16558056.0,"@ParisMatch comme il se doit ?? Pourquoi, Fran...",6.63335,46.5213,6c07f3233c333f95,1.611614e+17,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hollande
2012-04-20 11:29:39,1.933e+17,83416937.0,"“@Marion_Z : ""Les françaises sont belles mais ...",6.86580,46.5243,5a54953440569858,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hollande
2012-04-20 11:36:24,1.93302e+17,83416937.0,“@Marion_Z: @TwIttNTIN Les grosses vont voter ...,6.86579,46.5243,5a54953440569858,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hollande
2012-04-22 18:31:54,194131429242830848,465903153.0,Mtn imagine Hollande il va au Pays-Bas. A l'aé...,6.91641,46.6871,05e230dbb91aa51f,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hollande
2012-04-23 11:08:02,194382115780046848,56997480.0,Qg de campagne #hollande le changement avec fa...,5.99797,47.2433,24292aa21cb92cd8,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hollande
2012-04-26 19:22:32,1.95594e+17,88189861.0,"“@TheEconomist: #Hollande, Socialist likely to...",6.99062,46.2908,ae58be868b45d05f,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hollande
2012-04-26 20:14:49,195606883275522048,50337597.0,@SARKOZY_2012 président franchement il est pl...,6.60217,46.5868,fdeba41ff7f777ff,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hollande
2012-05-01 12:44:56,197305603679461376,83416937.0,"@nk_m traité sournoisement #Hollande de : ""Ren...",6.86625,46.5242,5a54953440569858,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hollande
2012-05-01 12:47:08,197306158472638466,83416937.0,"@nk_m traite sournoisement #Hollande de : ""Ren...",6.86582,46.5242,5a54953440569858,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hollande


In [42]:
gp.first()

,id,userId,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation
hashtag,,,,,,,,,,,,,,,,,,,
0,194731317739528192,472691649.0,Motivation pour faire du sport ? #0,6.00122,47.2623,24292aa21cb92cd8,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
002,3408881518645248,14823450.0,Neuchâtel #002 @ Beau Rivage hotel http://ins...,6.92951,46.9889,dfbf4ce81da97883,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0044cc,10709638593,14757865.0,Das #0044CC-Blau bingt laut Microsoft am beste...,7.23639,47.1359,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
007,6.15177e+16,120433.0,Eva Green ist super #007,7.47028,46.9503,e38a1a641d02f8db,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01bar,11764773097,5936932.0,@knowLED zwüsche central und helmhuus! #01bar,8.52552,47.3942,e91a16829a4387b1,1.176432e+10,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
021m,117517453269016576,10636452.0,My life: yesterday talking to directors of tou...,6.13757,46.1898,c3a6437e1b1a726d,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
031,200949621118337026,44444204.0,“@raeffuu: @raphiniert bärn baby isch eifach i...,9.09870,47.5684,08571a38aa5b52a6,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
039,202652893457813504,71528370.0,PM: &amp;quot;Resort Collina d&amp;#039;Oro&am...,8.91412,45.9676,4066be3eef8e635e,NaN,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0911,24184803107,17366149.0,und als Dank bauen sie am Ground zero eine Mus...,7.45351,46.9640,680cdd8d734a07a1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
len(dictionary)

28105

In [44]:
dictionary['hollande']

,text,hashtag,longitude,latitude,numberOfTweets
2012-01-23,"@ParisMatch comme il se doit ?? Pourquoi, Fran...",hollande,6.633350,46.52130,1
2012-04-20,"“@Marion_Z : ""Les françaises sont belles mais ...",hollande_$$$_hollande,6.865795,46.52430,2
2012-04-22,Mtn imagine Hollande il va au Pays-Bas. A l'aé...,hollande,6.916410,46.68710,1
2012-04-23,Qg de campagne #hollande le changement avec fa...,hollande,5.997970,47.24330,1
2012-04-26,"“@TheEconomist: #Hollande, Socialist likely to...",hollande_$$$_hollande,6.796395,46.43880,2
2012-05-01,"@nk_m traité sournoisement #Hollande de : ""Ren...",hollande_$$$_hollande,6.866035,46.52420,2
2012-05-02,#Hollande sait que s'il remonte la TVA sur la ...,hollande_$$$_hollande_$$$_hollande_$$$_holland...,6.601790,46.58650,7
2012-05-03,C'est @NicolasSarkozy qui gagnera car #Holland...,hollande_$$$_hollande,7.765290,46.67720,2
2012-05-04,"#Hollande est aimé des jeunes , du moins de ce...",hollande_$$$_hollande,6.224710,47.62670,2
2012-05-05,Dans ma ville l UMP ne communique que sur le d...,hollande_$$$_hollande_$$$_hollande_$$$_holland...,6.035700,47.23960,5


In [45]:
t = 0
for [h,df] in dictionary.items():
    t = t + 1
    if (t%10000 == 0):
        print(t)
    df['event'] = False
    if len(df) > 3:
        threshold = df['numberOfTweets'].mean() + 2 * df['numberOfTweets'].std()
        df['event'] = df.numberOfTweets.apply(lambda x: x>threshold)        

10000
20000


In [46]:
dictionary['hollande']

,text,hashtag,longitude,latitude,numberOfTweets,event
2012-01-23,"@ParisMatch comme il se doit ?? Pourquoi, Fran...",hollande,6.633350,46.52130,1,False
2012-04-20,"“@Marion_Z : ""Les françaises sont belles mais ...",hollande_$$$_hollande,6.865795,46.52430,2,False
2012-04-22,Mtn imagine Hollande il va au Pays-Bas. A l'aé...,hollande,6.916410,46.68710,1,False
2012-04-23,Qg de campagne #hollande le changement avec fa...,hollande,5.997970,47.24330,1,False
2012-04-26,"“@TheEconomist: #Hollande, Socialist likely to...",hollande_$$$_hollande,6.796395,46.43880,2,False
2012-05-01,"@nk_m traité sournoisement #Hollande de : ""Ren...",hollande_$$$_hollande,6.866035,46.52420,2,False
2012-05-02,#Hollande sait que s'il remonte la TVA sur la ...,hollande_$$$_hollande_$$$_hollande_$$$_holland...,6.601790,46.58650,7,True
2012-05-03,C'est @NicolasSarkozy qui gagnera car #Holland...,hollande_$$$_hollande,7.765290,46.67720,2,False
2012-05-04,"#Hollande est aimé des jeunes , du moins de ce...",hollande_$$$_hollande,6.224710,47.62670,2,False
2012-05-05,Dans ma ville l UMP ne communique que sur le d...,hollande_$$$_hollande_$$$_hollande_$$$_holland...,6.035700,47.23960,5,False


In [47]:
columns = dictionary['hollande'].columns
columns

Index(['text', 'hashtag', 'longitude', 'latitude', 'numberOfTweets', 'event'], dtype='object')

In [48]:
eventDf = pd.DataFrame(columns=columns)
eventDf

,text,hashtag,longitude,latitude,numberOfTweets,event


In [49]:
for h, df in dictionary.items():
    trueTemp = df[df['event'] == True]
    eventDf = eventDf.append(trueTemp)

In [50]:
eventDf.ix[0].hashtag

'tweetup_$$$_tweetup'

In [51]:
eventDf.head()

,text,hashtag,longitude,latitude,numberOfTweets,event
2012-05-11,... Het öpper um die 5i Zyt fürne #tweetup in ...,tweetup_$$$_tweetup,8.380045,47.2454,2.0,True
2010-07-30,"Unterwegs nach Basel, zuerst Swisscom Shop suc...",iphone4_$$$_iphone4_$$$_iphone4_$$$_iphone4_$$...,8.538800,47.3754,8.0,True
2012-05-21,sehr spät ubterwegs #gutenmorgen (@ Hohle Gass...,gutenmorgen_$$$_gutenmorgen,8.533060,47.5098,2.0,True
2011-09-08,"Hey, nice, kaum mehr als 600 Minuten bis #Feie...",feierabend_$$$_feierabend_$$$_feierabend,8.750000,47.5000,3.0,True
2012-04-20,@riiitou Bist Du Kader? #Zivilschutz_$$$_@rube...,zivilschutz_$$$_zivilschutz_$$$_zivilschutz,8.051940,47.3913,3.0,True


In [64]:
e_df = eventDf.copy()

In [65]:
e_df['date'] = e_df.index

In [70]:
e_df.index = [i for i in range (len(e_df))]

In [72]:
e_df
del e_df['event']

In [73]:
e_df.head(1)

,text,hashtag,longitude,latitude,numberOfTweets,date
0,... Het öpper um die 5i Zyt fürne #tweetup in ...,tweetup_$$$_tweetup,8.380045,47.2454,2.0,2012-05-11


In [74]:
e_gb = e_df.groupby(e_df.date)

In [115]:
'latitude:{0}, longitude:{1}, tweets: [{2}], number_of_tweets : {3} '.format('a', 'b', 'c', 'd')

'latitude:a, longitude:b, tweets: [c], number_of_tweets : d '

In [140]:
tweet_unit_template = '\"name\":\"{0}\", \"latitude\" : {1}, \"longitude\" : {2}, \
            \"tweets\": [{3}], \"number_of_tweets\" : {4}'

data_template = '\"data\": [{0}]' # put 'tweet_unit, ...'
event_template = '\"date\" : {0}, {1}' #put data to {1}

In [148]:
events = []
for date, df in e_gb:
        units = []
        for i in range (len(df)):
            ht = df.iloc[i]['hashtag'].split(delimiter)[0]
            lat = df.iloc[i]['latitude']
            lon = df.iloc[i]['longitude']
            t_num = df.iloc[i]['numberOfTweets']
            
            tw_list = df.iloc[i]['text'].replace(',', '').replace('\"', '').split(delimiter)
            tweets = ','.join('\"{0}\"'.format(tw) for tw in tw_list)

            units.append('{' + tweet_unit_template.format(ht, lat, lon, tweets, t_num) + '}')

        date_json = '\"date\": \"{0}\"'.format(date)
        data_json = data_template.format(','.join(units))
        curr_event = '{' + date_json + ',' + data_json + '}'
        events.append(curr_event)

final_json = '{\"events\": [' + ','.join(events) + ']}'

In [149]:
print(final_json[:200])

{"events": [{"date": "2010-03-20","data": [{"name":"sbb", "latitude" : 46.3526, "longitude" : 8.9733,             "tweets": ["Lugano—Zürich 40 min Verspätig son Gugus! #sbb","Läck bobby jetz müssen au


In [150]:
import json
with open('E:\\EPFL_STUDY\\AppliedDataAnalysis\\twitter-swisscom\\data.json', 'w') as f:
     json.dump(final_json, f)